In [1]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch 

import dataset
from model import AlexNet

In [2]:
device = torch.device('mps')
print(f" - device : {device}")

 - device : mps


In [3]:
transform = transforms.Compose([transforms.ToPILImage(),
                                transforms.Resize(227),
                                transforms.ToTensor()])

In [4]:
train_data = dataset.FashionDataset(transform=transform)
test_data = dataset.FashionDataset(name = 'fashion-mnist_test', transform=transform)

In [5]:
train_loader = DataLoader(train_data, batch_size=100)
test_loader = DataLoader(test_data, batch_size=100)

In [6]:
model = AlexNet().to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters()) 

In [9]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # enumberate() : 인덱스와 원소로 이루어진 튜플(tuple)을 만들어줌
        target = target.type(torch.LongTensor)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # 항상 backpropagation 하기전에 미분(gradient)을 zero로 만들어주고 시작해야 한다.
        output = model(data)
        loss = criterion(output, target) # criterion = loss_fn
        loss.backward() # Computes the gradient of current tensor w.r.t. graph leaves
        optimizer.step() # step() : 파라미터를 업데이트함
        if (batch_idx + 1) % 30 == 0:
            print("Train Epoch:{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [14]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)  # -> mean
        print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
        print('='*50)

In [15]:
epoch = 10
for epoch in range(1, epoch+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch:1 [2900/60000 (5%)]	Loss: 0.443842
Train Epoch:1 [5900/60000 (10%)]	Loss: 0.286798
Train Epoch:1 [8900/60000 (15%)]	Loss: 0.687962
Train Epoch:1 [11900/60000 (20%)]	Loss: 0.560333
Train Epoch:1 [14900/60000 (25%)]	Loss: 0.584882
Train Epoch:1 [17900/60000 (30%)]	Loss: 0.407818
Train Epoch:1 [20900/60000 (35%)]	Loss: 0.687556
Train Epoch:1 [23900/60000 (40%)]	Loss: 0.384985
Train Epoch:1 [26900/60000 (45%)]	Loss: 0.530314
Train Epoch:1 [29900/60000 (50%)]	Loss: 0.326464
Train Epoch:1 [32900/60000 (55%)]	Loss: 0.403286
Train Epoch:1 [35900/60000 (60%)]	Loss: 0.304617
Train Epoch:1 [38900/60000 (65%)]	Loss: 0.353906
Train Epoch:1 [41900/60000 (70%)]	Loss: 0.340622
Train Epoch:1 [44900/60000 (75%)]	Loss: 0.559474
Train Epoch:1 [47900/60000 (80%)]	Loss: 0.353269
Train Epoch:1 [50900/60000 (85%)]	Loss: 0.401234
Train Epoch:1 [53900/60000 (90%)]	Loss: 0.462912
Train Epoch:1 [56900/60000 (95%)]	Loss: 0.393021
Train Epoch:1 [59900/60000 (100%)]	Loss: 0.405416

Test set: Average loss